In [2]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from nltk.corpus import sentiwordnet as swn
import math
from math import exp, expm1, log, log10
import numpy as np
import turtle
import pandas as pd
from nltk.wsd import lesk
from pycorenlp import StanfordCoreNLP
from pywsd.lesk import adapted_lesk
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import sys, os


nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection"
]

[nltk_data] Downloading package stopwords to C:\Users\Farza
[nltk_data]     Nurifan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Farza Nurifan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Warming up PyWSD (takes ~10 secs)... took 5.071583271026611 secs.


In [2]:
from scipy.optimize import minimize
from scipy.spatial.distance import cdist

def geometric_medianX(points, method='auto', options={}):
    """
    Calculates the geometric median of an array of points.
    method specifies which algorithm to use:
        * 'auto' -- uses a heuristic to pick an algorithm
        * 'minimize' -- scipy.optimize the sum of distances
        * 'weiszfeld' -- Weiszfeld's algorithm
    """

    points = np.asarray(points)

    if len(points.shape) == 1:
        # geometric_median((0, 0)) has too much potential for error.
        # Did the user intend a single 2D point or two scalars?
        # Use np.median if you meant the latter.
        raise ValueError("Expected 2D array")

    if method == 'auto':
        if points.shape[1] > 2:
            # weiszfeld tends to converge faster in higher dimensions
            method = 'weiszfeld'
        else:
            method = 'minimize'

    return _methods[method](points, options)


def minimize_method(points, options={}):
    """
    Geometric median as a convex optimization problem.
    """

    # objective function
    def aggregate_distance(x):
        return cdist([x], points).sum()

    # initial guess: centroid
    centroid = points.mean(axis=0)

    optimize_result = minimize(aggregate_distance, centroid, method='COBYLA')

    return optimize_result.x


def weiszfeld_method(points, options={}):
    """
    Weiszfeld's algorithm as described on Wikipedia.
    """

    default_options = {'maxiter': 1000, 'tol': 1e-7}
    default_options.update(options)
    options = default_options

    def distance_func(x):
        return cdist([x], points)

    # initial guess: centroid
    guess = points.mean(axis=0)

    iters = 0

    while iters < options['maxiter']:
        distances = distance_func(guess).T

        # catch divide by zero
        # TODO: Wikipedia cites how to deal with distance 0
        distances = np.where(distances == 0, 1, distances)

        guess_next = (points/distances).sum(axis=0) / (1./distances).sum(axis=0)

        guess_movement = np.sqrt(((guess - guess_next)**2).sum())

        guess = guess_next

        if guess_movement <= options['tol']:
            break

        iters += 1

    return guess


_methods = {
    'minimize': minimize_method,
    'weiszfeld': weiszfeld_method,
}

In [4]:
def graph(radii, theta, key):
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     plt.plot([0,0],[-2,2])
#     plt.plot([-2,2],[0,0])
#     plt.axis([-1,1,-1,1])
    i=0
    a=[]
    b=[]
    rad=list(radii.values())
    theta1=list(theta.values())

    while(i<len(radii)):
        a.append(rad[i]*math.cos(theta1[i]))
        b.append(rad[i]*math.sin(theta1[i]))
        i+=1 
        
#     plt.scatter(a,b,label="circles",color="r",marker="o",s=10)
#     q = 0
#     rad2=list(radii.keys())
#     for i,j in zip(a,b):
#         ax.annotate('%s' %rad2[q], xy=(i,j), xytext=(15,0), textcoords='offset points')
#         q = q+1    
        
        
#     points = []
#     for x,y in zip(a,b):
#         points.append([x, y])

#     if(len(points) == 0):
#         points.append([0, 0])

#     senti_x, senti_y = geometric_medianX(points)
#     plt.scatter(senti_x,senti_y,label="circles",color="b",marker="o",s=10)
#     ax.annotate('%s' % '', xy=(senti_x,senti_y), xytext=(15,0), textcoords='offset points')
    
    
    
#     ax.add_artist(plt.Circle((0,0),1.0,color='b',fill=False))
#     plt.xlabel('Sentiment Strength')
#     plt.ylabel('Orientation')
#     plt.title(key)
#     plt.savefig("graph.png")
    
    return a, b

In [5]:
def analyse_file(key, lines):    
    radii = get_TDOC(lines, key)    
    return radii

def get_TDOC(lines, key):
    freq = {'Init': 0}              #Number of times context term occurs with key
    freq.clear()
    prohib = [''] #stopWords
    for line in lines:
        words = line.split(" ")
        if key in words:
            for context in words:
                flag=0
                for i in prohib:
                    if i == context:
                        flag=1
                        break
                if flag==0 and context!=key and context in op_set:
                    freq.setdefault(context, 0)
                    freq[context] = freq.get(context) + 1
                                           
    N = 0                           #Total Number of terms in Document
    for line in lines:
        words = line.split(" ")
        N += len(words)

    Nci = {'Init': 0}               #Total terms that occur with context term
    Nci.clear()
    for context in freq.keys():
        for line in lines:
            words = line.split(" ")
            if context in words:
                Nci.setdefault(context, 0)
                Nci[context] += len(words)

    radii = {'Init': 0}             #Get Radius of context term with TDOC formula
    radii.clear()
    
    df = pd.DataFrame(columns=['c', 'm', 'N', 'Nc', 'f', 'N/Nc', 'log(N/Nc)', 'fxlog(N/Nc)', '/4'])
    max_value = 0
    for term in freq.keys():
        radii[term] = (freq[term]*(log(N/Nci[term])))
        
        if radii[term] > max_value:
            max_value = radii[term]
        
    for term in freq.keys():
        radii[term] = radii[term]/max_value
        
        df = df.append({'c': term,
                'm': key,
                'N': N,
                'Nc': Nci[term],
                'f': freq[term],
                'N/Nc': "{0:.2f}".format(N/Nci[term]),
                'log(N/Nc)': "{0:.2f}".format(log(N/Nci[term])),
                'fxlog(N/Nc)': "{0:.2f}".format(freq[term]*(log(N/Nci[term]))),
                'normalisasi': "{0:.2f}".format((freq[term]*(log(N/Nci[term])))/max_value)
               }, ignore_index=True)
    
#     df.to_excel("tdoc.xlsx")
    return radii                    #Returns entire set of context terms related to key


In [30]:
def get_theta(key, sentences):
    scores = []
    for sentence in sentences:
        flag = True
        endscore = 0
#         if key in positive_lexicon:
#             endscore += 0.75
#         elif key in negative_lexicon:
#             endscore -= 0.75
            
#         if key == 'cramped':
#             endscore -= 0.2
#         elif key == 'unappealing':
#             endscore += 0.1
#         else:
#             endscore -= 0.2
        try:
            score = swn.senti_synset(key + '.a.1')
#             print(key)
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)
        except:
            print(key)
            
        words = sentence.split(' ')
        word_around = []
        for x in range(0, len(words)):
            try:
                if (words[x+1] == key) or (words[x+2] == key) or (words[x+3]== key) or (words[x+4]== key):
                    word_around.append(words[x])
                elif (words[x-1] == key):
                    word_around.append(words[x])
            except:
                pass

        for neg in negation:
            if neg in sentence:
#                 endscore *= (-1)
                break

        scores.append(endscore)
            
    final_score = np.average(scores)
    return np.pi * final_score

def prior_sentiment(radii, key, all_sentences):
    theta = {'Init': 0}
    theta.clear()
    for word in radii.keys():
        sentences = []
        for sentence in all_sentences:
            words = sentence.split(' ')
            if (word in words) and (key in words):
                sentences.append(sentence)
                
        filter = get_theta(word, sentences)            #if function returns 0 word does not exist in lexicon
        theta[word] = filter
        
    return theta

def senti(key, lines):
    radii = analyse_file(key, lines)
    theta = prior_sentiment(radii, key, lines)
    a, b = graph(radii, theta, key)
    points = []
    for x,y in zip(a,b):
        points.append([x, y])

    if(len(points) == 0):
        points.append([0, 0])
    senti_x, senti_y = geometric_medianX(points)
#     print(theta)
    return senti_x, senti_y
#     return theta

In [31]:
import string
def rem_punct(word):
    return word.replace('  ', ' ').strip().lower().translate(str.maketrans('', '', string.punctuation))

In [32]:
import pandas as pd

def run(tipe):
    df = pd.read_csv('Results/'+ tipe +'.csv')
    sf = pd.DataFrame(columns = ['opini','sentimen'])
    
    data = []
    for sent in df['review']:
        data.append(rem_punct(sent))

    opinions_all = []
    for opinions in df['opinion']:
        if type(opinions) != float:
            opinion = opinions.split('|')
            for o in opinion:
                if o == '-':
                    continue
                opinions_all.append(o)
    opinions_sorted = sorted(list(dict.fromkeys(opinions_all)))

    for opinion in opinions_sorted:
        a, b = senti(opinion, data)
        sentimen = ''
        if b > 0:
            sentimen = 'positive'
        else:
            sentimen = 'negative'
#         print(sentimen, '\t', opinion)
        sf = sf.append({'opini': opinion,
                        'sentimen': sentimen
                       }, ignore_index=True)
    sf.to_csv('Results/Opinion/'+ tipe + '-swn.csv')
    sf.to_excel('Results/Opinion/'+ tipe + '-swn.xlsx')

In [33]:
run('FOOD')
run('AMBIENCE')
run('PRICES')
run('SERVICE')

fairly
unusually
delight
enjoyed
outrageously
bargain
fun
perfection
reasonably
haze
reasonably
fairly
allergies
fun
reasonably
reasonably
enjoy
consistently
outrageously
allergies
enjoy
consistently
outrageously
unusually
gem
fun
delight
bargain
fun
reasonably
recommend
unusually
allergies
recommend
gem
allergies
fairly
fun
fairly
allergies
fairly
fun
enjoyed
gem
allergies
fun
perfection
recommend
unusually
fun
unusually
gem
fairly
recommend
delightfully
gem
love
vent
kills
gem
fun
gem
attraction
hang
fun
recommend
hate
enjoy
recommend
vent
love
complain
delight
love
dearth
recommend
love
delight
miraculously
recommend
fun
gimmick
dearth
gem
warmer
absurdly
work
recommend
enjoy
fairly
fun
exceeded
absurdly
work
enjoy
absurdly
work
enjoy
delight
bargain
fun
love
recommend
love
fairly
absurdly
work
fairly
absurdly
work
exceeded
delight
bargain
fun
incredibly
intimidate
absurdly
work
perfection
smile
perfection
haze
absurdly
work
smile
enjoy
enjoy
absurdly
work
died
incredibly
recommend


In [12]:
df = pd.read_csv('Results/'+ 'FOOD' +'.csv')

data = []
for sent in df['review']:
    data.append(rem_punct(sent))

In [13]:
a, b = senti('best', data)

expensive
great
ridiculous


In [36]:
b

0.003979001408701357

In [35]:
score = swn.senti_synset('good.a.1')

In [20]:
score.pos_score()

0.75

In [21]:
score.neg_score()

0.0

In [5]:
import random

positive = []
for i in range(0, 100):
    positive.append(positive_lexicon[random.randint(0,len(positive_lexicon))])
positive

['quicker',
 'insightfully',
 'lucid',
 'enviousness',
 'sensibly',
 'enchant',
 'breathtaking',
 'joyous',
 'unabashed',
 'backbone',
 'upbeat',
 'cushy',
 'lustrous',
 'beutifully',
 'brilliances',
 'energy-saving',
 'avidly',
 'complement',
 'reasonably',
 'luckier',
 'jubilation',
 'elevate',
 'comely',
 'attune',
 'hardier',
 'eloquently',
 'exciting',
 'innovative',
 'instrumental',
 'earnest',
 'impartial',
 'ameliorate',
 'manageable',
 'truthfully',
 'trendy',
 'cohesive',
 'ingenuity',
 'fruitful',
 'tingle',
 'freshest',
 'adaptable',
 'cleverly',
 'dashing',
 'mastery',
 'uplifting',
 'sensibly',
 'winning',
 'refunded',
 'blockbuster',
 'deserving',
 'heartily',
 'succeed',
 'fortune',
 'worthy',
 'remarkably',
 'razor-sharp',
 'autonomous',
 'breathtakingly',
 'adorable',
 'unassailable',
 'prestige',
 'spiritual',
 'acumen',
 'warmer',
 'supreme',
 'large-capacity',
 'gleefully',
 'overtaken',
 'encourage',
 'great',
 'earnestness',
 'tingle',
 'genial',
 'polite',
 'unp